In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from read_data import getData, LargeDataset, WindowedNormalizedDataset
from sklearn.preprocessing import StandardScaler
import gc


In [2]:
traindata, testData = getData("data")
traindata.shape, testData.shape

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (10000, 50, 110, 6)


((10000, 50, 110, 6), (2100, 50, 50, 6))

In [3]:
train_mean = np.mean(traindata, axis=(0, 1, 2))
train_std = np.std(traindata, axis=(0, 1, 2))
train_std = np.where(train_std == 0, 1.0, train_std)

In [4]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures, outfeatures = 2):
        super().__init__()
        self.layer1 = nn.Linear(in_features = infeatures, out_features = 16)
        # self.layer2 = nn.Linear(in_features = 16, out_features = 32)
        # self.layer3 = nn.Linear(in_features = 64, out_features = 64)
        # self.layer4 = nn.Linear(in_features = 64, out_features = 128)
        # self.layer5 = nn.Linear(in_features = 64, out_features = 128)
        # self.layer6 = nn.Linear(in_features = 256, out_features = 256)
        self.encoderlstm = nn.LSTM(input_size = 16, hidden_size = 32, num_layers = 2, batch_first = True, dropout = 0.2)

        self.pool = nn.AdaptiveAvgPool1d(10)

        self.decoderlstm = nn.LSTM(input_size = 32, hidden_size = 16, num_layers = 2, batch_first = True, dropout = 0.3)
        # self.layer7 = nn.Linear(in_features = 256, out_features = 256)
        # self.layer8 = nn.Linear(in_features = 128, out_features = 64)
        # self.layer9 = nn.Linear(in_features = 128, out_features = 64)
        # self.layer10 = nn.Linear(in_features = 64, out_features = 64)
        # self.layer11 = nn.Linear(in_features = 32, out_features = 16)
        self.layer12 = nn.Linear(in_features = 16, out_features = outfeatures)

    def forward(self, x):

        batch_size, channels, height, width = x.shape

        x = self.layer1(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        x = nn.GELU()(x)
        # x = self.layer2(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = nn.GELU()(x)
        # x = self.layer3(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = self.layer4(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = self.layer5(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = nn.GELU(x)
        # x = self.layer6(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)

        # print(x.shape, x.size(-1))
        x = x.view(batch_size, -1, x.size(-1))
        x, _ = self.encoderlstm(x)

        x = x.permute(0, 2, 1)  # [batch, 64, seq_len]
       

        x = self.pool(x)  # Forces output to [batch, channel, 10]

        x = x.permute(0, 2, 1)  # [batch, 64, seq_len]

        x, _ = self.decoderlstm(x)
        # print(x.shape)
        # x = x.view(batch_size, channels, height, width)

        # x = self.layer7(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = self.layer8(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = nn.GELU(x)
        # x = self.layer9(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = self.layer10(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = self.layer11(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        # x = nn.GELU()(x)
        x = self.layer12(x)
        # x = nn.functional.leaky_relu(x, negative_slope=0.01)
        x = nn.GELU()(x)

        return x

model = EncoderDecoderModel(6, 2)
test = torch.randn(10, 50, 110, 6)
out = model(test)
out.shape

torch.Size([10, 10, 2])

In [5]:
device = torch.device("cpu")

dataset = WindowedNormalizedDataset(traindata)
trainDataLoader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

model.to(device)

# Training setup
epochs = 100
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")

    for batchX, batchY in loop:
        batchX, batchY = batchX.to(device, non_blocking=True), batchY.to(device, non_blocking=True)
        output = model(batchX)
        loss = lossFn(output, batchY)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()

    avgLoss = runningLoss / len(trainDataLoader)

    # if each_epoch % 5 == 0:
    torch.save(model.state_dict(), f'./models/large_model_{avgLoss}.pth')
    print(f"Epoch {each_epoch + 1}, Training Loss: {avgLoss:.4f}")


Epoch [1/100]:  18%|█▊        | 3462/19063 [22:21<1:40:46,  2.58it/s]


KeyboardInterrupt: 